# Xente transactions dataset

# Xente is an e-commerce and financial service app serving 10,000+ customers in Uganda.

## Training data: 140K transactions recorded over November 2018 - February 2019 (3 months)

## Test data: 45K transactions recorded over 13 February 2019 to 14 March 2019 (1 month)


In [1]:
import pandas as pd

In [2]:
definitions = pd.DataFrame.from_csv('Xente_Variable_Definitions.csv')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [3]:
pd.set_option('display.max_colwidth', -1)
definitions

,Definition
Column Name,
TransactionId,Unique �transaction identifier on platform
BatchId,Unique number assigned to a batch of transactions for processing
AccountId,Unique number identifying the customer on platform
SubscriptionId,Unique number identifying the customer subscription
CustomerId,Unique identifier attached to Account
CurrencyCode,Country currency
CountryCode,Numerical geographical code of country
ProviderId,Source provider of Item �bought.
ProductId,Item name being bought.


In [4]:
data = pd.DataFrame.from_csv('training.csv')
data.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
TransactionId,,,,,,,,,,,,,,,
TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [5]:
def clear_numbers(dataframe):
    
    dataframe.AccountId = dataframe.AccountId.str.replace(r'AccountId_', '')
    dataframe.SubscriptionId = dataframe.SubscriptionId.str.replace(r'SubscriptionId_', '')
    dataframe.CustomerId = dataframe.CustomerId.str.replace(r'CustomerId_', '')
    dataframe.ProviderId = dataframe.ProviderId.str.replace(r'ProviderId_', '')
    dataframe.ProductId = dataframe.ProductId.str.replace(r'ProductId_', '')
    dataframe.ChannelId = dataframe.ChannelId.str.replace(r'ChannelId_', '')
    dataframe.BatchId = dataframe.BatchId.str.replace(r'BatchId_', '')
    
    return dataframe

def clear_time(dataframe):
    dataframe.TransactionStartTime = dataframe.TransactionStartTime.replace(r'T',' ', regex=True)
    dataframe.TransactionStartTime = dataframe.TransactionStartTime.replace(r'Z','', regex=True)
    
    return dataframe

def clear_df(dataframe):
    dataframe = clear_numbers(dataframe)
    dataframe = clear_time(dataframe)
    
    dataframe = dataframe.drop(['CurrencyCode', 'CountryCode', 'Amount'], axis=1)
    
    return dataframe

def add_category_means(dataframe):
    product_category_means = dataframe.groupby(['ProductCategory']).mean().Value
    product_category_means = pd.DataFrame(product_category_means)
    
    dataframe['copy_index'] = dataframe.index
    dataframe = pd.merge(dataframe, product_category_means, how='left', on=['ProductCategory'])
    dataframe = dataframe.rename(columns={'Value_x':'Value', 'Value_y':'Category_mean_value'})
    
    return dataframe

def add_tx_on_category_means(dataframe):
    dataframe['ValueOnCategoryMean'] = dataframe.Value / dataframe['Category_mean_value']
    
    return dataframe

def add_txs_count(dataframe):
    account_txs = dataframe.groupby(['AccountId']).count().BatchId
    account_txs = pd.DataFrame(account_txs)
    account_txs = account_txs.rename(columns={'AccountId':'Value'})
    
    dataframe = pd.merge(dataframe,account_txs, on=['AccountId'], how="left")
    dataframe = dataframe.rename(columns={'BatchId_x':'BatchId', 'BatchId_y':'txs_count'})
    
    return dataframe

def get_dummies(dataframe):
    dataframe = pd.get_dummies(dataframe, columns=['ProductCategory'])
    
    return dataframe

def prepare_df(dataframe):
    
    dataframe = clear_df(dataframe)
    dataframe = add_category_means(dataframe)
    dataframe = add_tx_on_category_means(dataframe)
    dataframe = add_txs_count(dataframe)
    dataframe = get_dummies(dataframe)
    
    return dataframe

In [6]:
data = prepare_df(data)

In [7]:
pd.options.display.max_columns = 500

data.head()

,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ChannelId,Value,TransactionStartTime,PricingStrategy,FraudResult,copy_index,Category_mean_value,ValueOnCategoryMean,txs_count,ProductCategory_airtime,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill
0,36123,3957,887,4406,6,10,3,1000,2018-11-15 02:18:49,2,0,TransactionId_76871,6049.799609,0.165295,66,1,0,0,0,0,0,0,0,0
1,15642,4841,3829,4406,4,6,2,20,2018-11-15 02:19:08,2,0,TransactionId_73770,12734.467570,0.001571,30893,0,0,1,0,0,0,0,0,0
2,53941,4229,222,4683,6,1,3,500,2018-11-15 02:44:21,2,0,TransactionId_26203,6049.799609,0.082647,2,1,0,0,0,0,0,0,0,0
3,102363,648,2185,988,1,21,3,21800,2018-11-15 03:32:55,2,0,TransactionId_380,20946.692188,1.040737,26,0,0,0,0,0,0,0,0,1
4,38780,4841,3829,988,4,6,2,644,2018-11-15 03:34:21,2,0,TransactionId_28195,12734.467570,0.050571,30893,0,0,1,0,0,0,0,0,0


In [8]:
data.head()

,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ChannelId,Value,TransactionStartTime,PricingStrategy,FraudResult,copy_index,Category_mean_value,ValueOnCategoryMean,txs_count,ProductCategory_airtime,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_other,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill
0,36123,3957,887,4406,6,10,3,1000,2018-11-15 02:18:49,2,0,TransactionId_76871,6049.799609,0.165295,66,1,0,0,0,0,0,0,0,0
1,15642,4841,3829,4406,4,6,2,20,2018-11-15 02:19:08,2,0,TransactionId_73770,12734.467570,0.001571,30893,0,0,1,0,0,0,0,0,0
2,53941,4229,222,4683,6,1,3,500,2018-11-15 02:44:21,2,0,TransactionId_26203,6049.799609,0.082647,2,1,0,0,0,0,0,0,0,0
3,102363,648,2185,988,1,21,3,21800,2018-11-15 03:32:55,2,0,TransactionId_380,20946.692188,1.040737,26,0,0,0,0,0,0,0,0,1
4,38780,4841,3829,988,4,6,2,644,2018-11-15 03:34:21,2,0,TransactionId_28195,12734.467570,0.050571,30893,0,0,1,0,0,0,0,0,0


In [9]:
y = pd.Series(data.FraudResult)

In [10]:
data_index = data.copy_index

In [11]:
data = data.drop(['copy_index','TransactionStartTime', 'BatchId','AccountId','SubscriptionId','CustomerId','Value','FraudResult','Category_mean_value'], axis=1)


In [12]:
import numpy as np

parameters = [{'n_estimators': [10, 15], 'max_features': np.arange(3, 5), 'max_depth': np.arange(4, 7)}]

In [13]:
from sklearn.ensemble import RandomForestClassifier

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
grid_model = GridSearchCV(estimator=RandomForestClassifier(random_state=100), param_grid=parameters, scoring='roc_auc', cv=3)


In [16]:
grid_model.fit(data, y)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=100, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [10, 15], 'max_features': array([3, 4]), 'max_depth': array([4, 5, 6])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [17]:
grid_model.best_params_

{'max_depth': 5, 'max_features': 3, 'n_estimators': 15}

In [18]:
test_data = pd.DataFrame.from_csv('test.csv')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


In [19]:
test_data.head()

,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
TransactionId,,,,,,,,,,,,,,
TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-13T10:01:40Z,4
TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2000.0,2000,2019-02-13T10:02:12Z,2
TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-13T10:02:30Z,2
TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,3000.0,3000,2019-02-13T10:02:38Z,4
TransactionId_22140,BatchId_29804,AccountId_4841,SubscriptionId_3829,CustomerId_3105,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-60.0,60,2019-02-13T10:02:58Z,2


In [20]:
test_data = prepare_df(test_data)


In [21]:
test_data.head()

,BatchId,AccountId,SubscriptionId,CustomerId,ProviderId,ProductId,ChannelId,Value,TransactionStartTime,PricingStrategy,copy_index,Category_mean_value,ValueOnCategoryMean,txs_count,ProductCategory_airtime,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_retail,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill
0,35028,2441,4426,2857,5,3,3,1000,2019-02-13 10:01:40,4,TransactionId_50600,7157.965875,0.139704,6,1,0,0,0,0,0,0,0,0
1,45139,3439,2643,3874,5,15,3,2000,2019-02-13 10:02:12,2,TransactionId_95109,10895.748848,0.183558,39,0,0,1,0,0,0,0,0,0
2,74887,4841,3829,2857,4,6,2,50,2019-02-13 10:02:30,2,TransactionId_47357,10895.748848,0.004589,11441,0,0,1,0,0,0,0,0,0
3,11025,2685,4626,3105,5,10,3,3000,2019-02-13 10:02:38,4,TransactionId_28185,7157.965875,0.419113,35,1,0,0,0,0,0,0,0,0
4,29804,4841,3829,3105,4,6,2,60,2019-02-13 10:02:58,2,TransactionId_22140,10895.748848,0.005507,11441,0,0,1,0,0,0,0,0,0


In [22]:
test_data_index = test_data.copy_index

In [23]:
test_data = test_data.drop(['copy_index','TransactionStartTime', 'BatchId','AccountId','SubscriptionId','CustomerId','Value','Category_mean_value'], axis=1)


In [24]:
pd.options.display.max_columns = 500

test_data.head()

,ProviderId,ProductId,ChannelId,PricingStrategy,ValueOnCategoryMean,txs_count,ProductCategory_airtime,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,ProductCategory_retail,ProductCategory_ticket,ProductCategory_transport,ProductCategory_tv,ProductCategory_utility_bill
0,5,3,3,4,0.139704,6,1,0,0,0,0,0,0,0,0
1,5,15,3,2,0.183558,39,0,0,1,0,0,0,0,0,0
2,4,6,2,2,0.004589,11441,0,0,1,0,0,0,0,0,0
3,5,10,3,4,0.419113,35,1,0,0,0,0,0,0,0,0
4,4,6,2,2,0.005507,11441,0,0,1,0,0,0,0,0,0


In [25]:
test_pred = grid_model.predict_proba(test_data)[:,1]
test_pred = pd.Series(test_pred)

In [26]:
test_pred

0        0.000302
1        0.003797
2        0.000032
3        0.000142
4        0.000032
5        0.000285
6        0.000032
7        0.000758
8        0.000032
9        0.043980
10       0.000168
11       0.000263
12       0.000032
13       0.000210
14       0.000285
15       0.000210
16       0.000032
17       0.000032
18       0.000184
19       0.000032
20       0.007364
21       0.000274
22       0.000032
23       0.001686
24       0.000032
25       0.000250
26       0.000048
27       0.000183
28       0.000032
29       0.000349
           ...   
44989    0.000338
44990    0.001121
44991    0.000338
44992    0.000048
44993    0.001121
44994    0.000338
44995    0.001121
44996    0.000338
44997    0.001121
44998    0.001121
44999    0.001121
45000    0.000338
45001    0.001121
45002    0.000338
45003    0.004462
45004    0.000340
45005    0.000338
45006    0.001478
45007    0.000338
45008    0.000340
45009    0.001121
45010    0.000338
45011    0.001121
45012    0.000338
45013    0

In [34]:
predictions = pd.DataFrame(test_data_index)
predictions['FraudResult'] = np.where(test_pred > 0.3, 1, 0)

In [35]:
predictions = predictions.set_index('copy_index')

In [36]:
predictions.index.name = 'Transaction_id'

In [37]:
predictions.describe()

,FraudResult
count,45019.000000
mean,0.001444
std,0.037971
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [38]:
predictions.to_csv('submission.csv')

In [ ]:
predictions = predictions.index.rename('TransactionId')

In [ ]:
predictions

In [ ]:
predictions.loc[predictions['FraudResult'] == True]

In [ ]:
pd.options.display.float_format = '{:20,.2f}'.format # ЗОЛОТО
data.describe()

In [ ]:
sample = pd.DataFrame.from_csv('sample_submission.csv')

In [ ]:
sample.head()